In [4]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) # Add the parent directory to the path since we work with notebooks
from typing import List
import spacy
from spacy.util import minibatch, compounding
from spacy.training.example import Example
import random

In [5]:
#-----------------------------------------
# CUSTOMIZING SPACY MODEL. NLP TRAINING.
# source: https://www.machinelearningplus.com/nlp/training-custom-ner-model-in-spacy/#:~:text=8.-,Training%20completely%20new%20entity%20type%20in%20spaCy,-In%20previous%20section
TRAIN_DATA = [
("The AKHMAT Special Forces conducted a daring operation in the eastern region.",
{"entities": [(4, 25, "MILITARY_UNIT")]}),
("Soldiers from the 24th Mechanized Brigade fortified their positions near Bakhmut.",
{"entities": [(18, 41, "MILITARY_UNIT")]}),
("The 128th Mountain Assault Brigade is renowned for its resilience in mountainous warfare.",
{"entities": [(4, 34, "MILITARY_UNIT")]}),
("During the counteroffensive, the 92nd Mechanized Brigade advanced significantly.",
{"entities": [(32, 56, "MILITARY_UNIT")]}),
("The 1st Presidential Brigade provides critical support during ceremonial events and combat operations.",
{"entities": [(4, 28, "MILITARY_UNIT")]}),
("Engineers from the 211th Pontoon Bridge Brigade built a crossing over the Dnipro River.",
{"entities": [(19, 47, "MILITARY_UNIT")]}),
("The 79th Air Assault Brigade successfully repelled an enemy attack.",
{"entities": [(4, 29, "MILITARY_UNIT")]}),
("Troops of the 57th Motorized Infantry Brigade maintained defensive lines under heavy shelling.",
{"entities": [(14, 45, "MILITARY_UNIT")]}),
("The 3rd Tank Brigade was deployed to reinforce key positions in the south.",
{"entities": [(4, 21, "MILITARY_UNIT")]}),
("A reconnaissance mission was completed by the 73rd Naval Special Operations Center.",
{"entities": [(46, 82, "MILITARY_UNIT")]}),
("The 30th Mechanized Brigade has played a pivotal role in recent battles.",
{"entities": [(4, 27, "MILITARY_UNIT")]}),
("The 36th Marine Brigade conducted amphibious training near the Black Sea coast.",
{"entities": [(4, 24, "MILITARY_UNIT")]}),
("Members of the 503rd Separate Marine Battalion were awarded for their bravery.",
{"entities": [(15, 46, "MILITARY_UNIT")]}),
("The 10th Mountain Assault Brigade launched a pre-dawn offensive.",
{"entities": [(4, 33, "MILITARY_UNIT")]}),
("A strategic river crossing was established by the 12th Engineer Brigade.",
{"entities": [(50, 71, "MILITARY_UNIT")]}),
("The 17th Tank Brigade spearheaded the breakthrough on the western flank.",
{"entities": [(4, 22, "MILITARY_UNIT")]}),
("Soldiers from the 58th Motorized Infantry Brigade demonstrated exceptional courage.",
{"entities": [(18, 49, "MILITARY_UNIT")]}),
("The 59th Motorized Brigade was involved in heavy fighting in the Donetsk region.",
{"entities": [(4, 26, "MILITARY_UNIT")]}),
("Combat engineers of the 14th Mechanized Brigade repaired critical infrastructure.",
{"entities": [(24, 47, "MILITARY_UNIT")]}),
("The 72nd Mechanized Brigade has consistently held its ground despite enemy advances.",
{"entities": [(4, 27, "MILITARY_UNIT")]}),
("The 93rd Mechanized Brigade is celebrated for its key contributions to recent successes.",
{"entities": [(4, 27, "MILITARY_UNIT")]}),
("During an assault, the 80th Air Assault Brigade captured key objectives.",
{"entities": [(23, 48, "MILITARY_UNIT")]}),
("The 81st Airmobile Brigade completed a high-risk deployment to the frontlines.",
{"entities": [(4, 27, "MILITARY_UNIT")]}),
("Paratroopers from the 25th Airborne Brigade neutralized multiple enemy positions.",
{"entities": [(21, 44, "MILITARY_UNIT")]}),
("The 1st Tank Brigade reinforced defensive positions around Kharkiv.",
{"entities": [(4, 20, "MILITARY_UNIT")]}),
("The 18th Coastal Defense Battalion conducted patrols along the coastline.",
{"entities": [(4, 34, "MILITARY_UNIT")]}),
("A successful counterattack was led by the 93rd Mechanized Brigade.",
{"entities": [(42, 65, "MILITARY_UNIT")]}),
("The 77th Airmobile Brigade executed an aerial insertion into hostile territory.",
{"entities": [(4, 27, "MILITARY_UNIT")]}),
("Special forces of the 140th Special Operations Unit eliminated high-value targets.",
{"entities": [(22, 51, "MILITARY_UNIT")]}),
("The 44th Artillery Brigade provided long-range fire support during the operation.",
{"entities": [(4, 26, "MILITARY_UNIT")]}),
("The 38th Naval Infantry Brigade secured key points along the riverbank.",
{"entities": [(4, 31, "MILITARY_UNIT")]}),
("Soldiers of the 43rd Heavy Artillery Brigade targeted enemy supply lines.",
{"entities": [(16, 44, "MILITARY_UNIT")]}),
("The 82nd Air Assault Brigade is trained for rapid response operations.",
{"entities": [(4, 29, "MILITARY_UNIT")]}),
("Recon elements from the 75th Ranger Division gathered critical intelligence.",
{"entities": [(24, 44, "MILITARY_UNIT")]}),
("Engineers of the 21st Separate Engineering Battalion constructed defensive barriers.",
{"entities": [(16, 52, "MILITARY_UNIT")]}),
("The 35th Marine Brigade carried out coastal operations under enemy fire.",
{"entities": [(4, 24, "MILITARY_UNIT")]}),
("The 45th Air Defense Brigade intercepted incoming aerial threats.",
{"entities": [(4, 28, "MILITARY_UNIT")]}),
("The 50th Signal Brigade ensured uninterrupted communication across units.",
{"entities": [(4, 23, "MILITARY_UNIT")]}),
("The 55th Artillery Brigade delivered precision strikes against enemy fortifications.",
{"entities": [(4, 26, "MILITARY_UNIT")]}),
("Troops from the 19th Rocket Artillery Brigade launched coordinated attacks.",
{"entities": [(16, 45, "MILITARY_UNIT")]}),
("The 41st Engineering Division played a crucial role in clearing minefields.",
{"entities": [(4, 30, "MILITARY_UNIT")]}),
("During the operation, the GRUT Mechanized Unit secured a vital supply route.",
{"entities": [(26, 47, "MILITARY_UNIT")]}),
("The 39th Infantry Brigade conducted joint exercises with allied forces.",
{"entities": [(4, 25, "MILITARY_UNIT")]}),
("A swift raid was conducted by the 31st Mechanized Brigade in the northern sector.",
{"entities": [(34, 57, "MILITARY_UNIT")]}),
("The 22nd Mechanized Brigade has maintained control of key areas in the east.",
{"entities": [(4, 27, "MILITARY_UNIT")]}),
("The 29th Motorized Brigade intercepted an enemy convoy near the border.",
{"entities": [(4, 26, "MILITARY_UNIT")]}),
("The 67th Infantry Brigade participated in the defense of key urban centers.",
{"entities": [(4, 25, "MILITARY_UNIT")]}),
("The 101st Territorial Defense Brigade has been critical in protecting civilians.",
{"entities": [(4, 37, "MILITARY_UNIT")]}),
("The 16th Tank Brigade moved swiftly to reinforce embattled positions.",
{"entities": [(4, 21, "MILITARY_UNIT")]}),
("A crucial supply line was secured by the 47th Mechanized Brigade.",
{"entities": [(41, 64, "MILITARY_UNIT")]}),
]

In [6]:
for e in range(0, len(TRAIN_DATA)):
    start = TRAIN_DATA[e][1]["entities"][0][0]
    end = TRAIN_DATA[e][1]["entities"][0][1]
    print(f"\n-----------------------------------------------------\nAt index {e} in list the military unit is: {TRAIN_DATA[e][0][start:end]}")


-----------------------------------------------------
At index 0 in list the military unit is: AKHMAT Special Forces

-----------------------------------------------------
At index 1 in list the military unit is: 24th Mechanized Brigade

-----------------------------------------------------
At index 2 in list the military unit is: 128th Mountain Assault Brigade

-----------------------------------------------------
At index 3 in list the military unit is:  92nd Mechanized Brigade

-----------------------------------------------------
At index 4 in list the military unit is: 1st Presidential Brigade

-----------------------------------------------------
At index 5 in list the military unit is: 211th Pontoon Bridge Brigade

-----------------------------------------------------
At index 6 in list the military unit is: 79th Air Assault Brigade 

-----------------------------------------------------
At index 7 in list the military unit is: 57th Motorized Infantry Brigade

-----------------

In [7]:
def train_spacy_ner_model(new_label: str, train_data: List[tuple], lang: str="en"):
    if new_label != train_data[0][1]["entities"][0][2]:
        raise ValueError("The given NER label does not fit your training data!")
    if lang == "en":
        model = "en_core_web_sm"
    else:
        model = "de_core_news_sm"
    # load spacy language model
    nlp=spacy.load(model) 
    # load the ner component
    ner=nlp.get_pipe('ner')
    # previous labels
    original_labels = ner.labels
    # add the new label to the namespace
    ner.add_label(new_label)
    # get the optimizer from the spacy training
    optimizer = nlp.resume_training()
    # List of pipes you want to train
    pipes_to_be_trained = ["ner"]
    # List of pipes which should remain unaffected in training
    disabled_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipes_to_be_trained]
    print("disabled pipes are: {}".format(disabled_pipes))
    
    #------------------------------------------------------
    # Training
    # Begin training by disabling other pipeline components
    with nlp.disable_pipes(*disabled_pipes) :
        sizes = compounding(1.0, 4.0, 1.001)
        # Training for 30 iterations     
        for r in range(30):
            # shuffle examples before training
            random.shuffle(train_data)
            # batch up the examples using spaCy's minibatch
            batches = minibatch(train_data, size=sizes)
            # ictionary to store losses
            losses = {}
            # source: https://stackoverflow.com/questions/66675261/how-can-i-work-with-example-for-nlp-update-problem-with-spacy3-0
            for batch in batches:
                for text, annotations in batch:
                    # create example instance
                    doc = nlp.make_doc(text=text)
                    example = Example.from_dict(doc, annotations)
                    # Calling update() over the iteration
                    nlp.update([example], sgd=optimizer, drop=0.35, losses=losses)
                    print("Losses", losses)
    # labels after retraining
    retrained_labels = nlp.get_pipe("ner").labels
    # Check of previous labels have been preserved
    missing_labels = set(original_labels) - set(retrained_labels)
    if missing_labels:
        print(f"Warning: Missing labels after retraining - {missing_labels}")
    MODEL_DIR = os.path.join(os.getcwd(), 'models/custom_spacy_model')
    if not os.path.exists(MODEL_DIR):
        os.makedirs(MODEL_DIR, exist_ok=True)
    nlp.meta['name'] = 'custom_ner'  # rename model
    nlp.to_disk(MODEL_DIR)
    print("Saved model to:{}".format(MODEL_DIR))
    return nlp

In [8]:
nlp_test = train_spacy_ner_model(new_label="MILITARY_UNIT", train_data=TRAIN_DATA, lang="en")

disabled pipes are: ['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer']
Losses {'ner': 2.20579607862324}
Losses {'ner': 5.619978098933292}
Losses {'ner': 6.9620392235825665}
Losses {'ner': 10.369605279736305}
Losses {'ner': 13.834919944676022}
Losses {'ner': 17.502080357035805}
Losses {'ner': 18.731608526537023}
Losses {'ner': 20.837755353888248}


/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "The 3rd Tank Brigade was deployed to reinforce key..." with entities "[(4, 21, 'MILITARY_UNIT')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Engineers of the 21st Separate Engineering Battali..." with entities "[(16, 52, 'MILITARY_UNIT')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


Losses {'ner': 20.837764740501562}
Losses {'ner': 22.697095939443713}
Losses {'ner': 24.3728106329939}
Losses {'ner': 26.114144330345187}
Losses {'ner': 27.823161173485573}
Losses {'ner': 29.04880758406725}
Losses {'ner': 30.333883526651924}
Losses {'ner': 30.333938444998292}
Losses {'ner': 32.08519642453165}
Losses {'ner': 33.27163119247749}
Losses {'ner': 34.78140193256476}
Losses {'ner': 36.55699850137198}


/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "The 81st Airmobile Brigade completed a high-risk d..." with entities "[(4, 27, 'MILITARY_UNIT')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "The 79th Air Assault Brigade successfully repelled..." with entities "[(4, 29, 'MILITARY_UNIT')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


Losses {'ner': 37.67722963675526}
Losses {'ner': 38.81123640628373}
Losses {'ner': 40.25906743762133}
Losses {'ner': 41.65394835107405}
Losses {'ner': 41.654427429766024}
Losses {'ner': 42.95290453358046}
Losses {'ner': 42.987556935924346}
Losses {'ner': 44.09882342508281}
Losses {'ner': 44.09911211396121}
Losses {'ner': 45.280408006437334}
Losses {'ner': 45.41868377211936}


/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "During the operation, the GRUT Mechanized Unit sec..." with entities "[(26, 47, 'MILITARY_UNIT')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "The 77th Airmobile Brigade executed an aerial inse..." with entities "[(4, 27, 'MILITARY_UNIT')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "The 36th Marine Brigade conducte

Losses {'ner': 46.528295227987655}
Losses {'ner': 48.515496703938595}
Losses {'ner': 48.518288856777694}
Losses {'ner': 49.5995103352226}
Losses {'ner': 49.60000109607947}
Losses {'ner': 50.46214027900535}
Losses {'ner': 51.52866834112947}
Losses {'ner': 52.50170005397797}
Losses {'ner': 52.50177634410969}
Losses {'ner': 53.83296796673466}
Losses {'ner': 54.862826669983704}


/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "During the counteroffensive, the 92nd Mechanized B..." with entities "[(32, 56, 'MILITARY_UNIT')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "The 82nd Air Assault Brigade is trained for rapid ..." with entities "[(4, 29, 'MILITARY_UNIT')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "The 41st Engineering Division pl

Losses {'ner': 69.57892755991232}
Losses {'ner': 70.44542198450556}
Losses {'ner': 83.17841561618745}
Losses {'ner': 86.99850108979506}
Losses {'ner': 92.34044468405045}
Losses {'ner': 98.46039475977534}
Losses {'ner': 100.3072768452665}
Losses {'ner': 102.39534480835799}
Losses {'ner': 4.860755641811604}
Losses {'ner': 6.9445895749201405}
Losses {'ner': 7.6636920904129155}
Losses {'ner': 11.447509452937911}
Losses {'ner': 15.88849936501908}
Losses {'ner': 18.567388496478102}
Losses {'ner': 19.851482483581076}
Losses {'ner': 23.211053570082573}
Losses {'ner': 27.997585388005575}
Losses {'ner': 31.434211599769412}
Losses {'ner': 32.72620137788357}
Losses {'ner': 34.232456168467195}
Losses {'ner': 35.41662262075238}
Losses {'ner': 37.57282780824585}
Losses {'ner': 41.11177581374318}
Losses {'ner': 42.464437604342265}
Losses {'ner': 43.36681859463965}
Losses {'ner': 44.316485129202725}
Losses {'ner': 44.75051405724534}
Losses {'ner': 44.81660322882303}
Losses {'ner': 45.7884087408639}
Los

In [9]:
test = "Bloomberg L.P. reported yesterday at 2 p.m. in New York at the UN Conference Center that in Kherson the AKHMAT special forces and the 99th Special Operations Battalion recaptured several strategic parts in the Kherson region and destroyed many vehicles of the Russion 236th infantery division."
doc = nlp_test(test)
print(f"All available doc entities:\n{doc.ents}")
named_entities = [ent.text for ent in doc.ents if ent.label_ in ["PERSON", "ORG", "GPE", "DATE", "TIME", "LOC", "MILITARY_UNIT"]]
for ent in doc.ents:
    print(f"ent.label: {ent.label_} >>> ent.text: {ent.text}")

All available doc entities:
(New York, UN Conference Center, AKHMAT special forces, 99th Special Operations Battalion)
ent.label: GPE >>> ent.text: New York
ent.label: MILITARY_UNIT >>> ent.text: UN Conference Center
ent.label: MILITARY_UNIT >>> ent.text: AKHMAT special forces
ent.label: MILITARY_UNIT >>> ent.text: 99th Special Operations Battalion


In [13]:
from spacy import displacy

text = "When Sebastian Thrun started working on self-driving cars at Google in 2007, few people outside of the company took him seriously."

nlp2 = spacy.load("/workspaces/20241123_RAG_System/models/custom_spacy_model")
doc2 = nlp2(text)
colors = {"ORG": "linear-gradient(90deg, #aa9cfc, #fc9ce7)"}
options = {"ents": ["PERSON", "ORG", "DATE"], "colors": colors}
displacy.render(doc2, style="ent", options=options)

In [14]:
# SOURCE: https://spacy.io/usage/linguistic-features#setting-entities
from spacy.tokens import Span

nlp = spacy.load("en_core_web_sm")
doc = nlp("fb is hiring a new vice president called of global policy. Joseph Musterman will probably apply from Bloomberg L.P.")
ents = [(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents]
print('Before', ents)
# The model didn't recognize "fb" as an entity :(

# Create a span for the new entity
fb_ent = Span(doc, 0, 1, label="ORG")
print("New ent:{}".format(fb_ent))
orig_ents = list(doc.ents)

# Option 1: Modify the provided entity spans, leaving the rest unmodified
doc.set_ents([fb_ent], default="unmodified")

# Option 2: Assign a complete list of ents to doc.ents
doc.ents = orig_ents + [fb_ent]

ents = [(e.text, e.start, e.end, e.label_) for e in doc.ents]
print('After', ents)
# [('fb', 0, 1, 'ORG')] 🎉

Before [('Joseph Musterman', 59, 75, 'PERSON'), ('Bloomberg L.P.', 101, 115, 'ORG')]
New ent:fb
After [('fb', 0, 1, 'ORG'), ('Joseph Musterman', 12, 14, 'PERSON'), ('Bloomberg L.P.', 18, 20, 'ORG')]
